# Dynamic programming

Dynamic programming is an algorithmic method for solving optimization problems, i.e. roughly, for solving problems where one wants to find a value $x \in A$ thats maximizes a function $f : A \rightarrow \mathbb{R}$ --- i.e. a value $x \in A$ s.t. $\forall y \in A .\, f(y) \leq f(x)$.

In the dynamic programming setting, one sets up the search space in such a way that:
 - The optimal solution to a problem can be built easily from the optimal solution for subproblems of the same type; this is called *optimal substructure*,
 - Subproblems are overlapping.

We exemplified the dynamic programming technique over a simple problem.

## Maximum path sum in a triangle

Given a pyramid of numbers like the one below, the goal is to find the maximum sum along paths descending from the root of the triangle to the base.

```
     3
    7 4
   2 4 6
  9 5 9 3
```

One solution to solve this problem is to try all paths. However, even for medium size triangles, this is not possible in practive. Indeed, for a triangle of height $n$, there exist $2^{n-1}$ paths. For example, for a triangle of size $100$, if one could check one billion paths per second, it would take more than $20000000000000$ years to check all of them.

Here, dynamic programming can come to help. In dynamic programming, we first have to find the *optimal substructure* of our problem. Here, the structure is quite easy to see, as exemplified below:

<img src="files/subtriangles.png" style="height: 100px">

For finding the maximal path of the triangle, it is sufficient to find a maximum path in resp. the red and blue sub-triangles, and then to select the best of the two and to add (value of) the root to it. For instance, a maximal path in the red triangle is `7-4-9` (with sum `20`) and a maximal path in the blue triangle is `4-6-9` (of sum `19`). The former having a bigger sum that the latter, we select it and add `3` to it, obtaining the maximum path `3-7-4-9` (of sum `23`).

More formally, a number in the pyramid is indexed by its lines number (lines are numberer from top to bottom, starting at `0`) and its position in its line. For example, the root `3` of the pyramid has position `(0, 0)`, whereas the unique `7` of the pyramid has position `(1, 0)`.

For a position `p`, we denote by $\lambda_L(p)$ (resp. $\lambda_R(p)$) the position, when it exists, of the number below and on the left (resp. right) of `p`. I.e., $\lambda_L((x, y)) = (x+1, y)$ and $\lambda_R((x, y)) = (x+1, y+1)$.


We also denote by $\nu(p)$ the number located at `p` and by $\pi(p)$ the sum of a maximal path of the sub-triangle rooted at `p`. When `p` is a leaf position (i.e. a position that belongs to the last line of the triangle), we have $\pi(p) = \nu(p)$. Otherwise, assuming that `p = (x, y)`, we have

$$\pi(p) = \max(\pi(\lambda_L(p)) + \pi(\lambda_R(p))) + \nu(p) .$$

Such a recursive structure leads to a direct implementation using recursion:

In [1]:
EXAMPLE = [
  [3],
  [7, 4],
  [2, 4, 6],
  [9, 5, 9, 3],
]

def sum_td(tg, depth, index):
    if depth+1 == len(tg):
        return tg[depth][index]
    aout = max(sum_td(tg, depth+1, index), sum_td(tg, depth+1, index+1))
    return tg[depth][index] + aout

print(sum_td(EXAMPLE, 0, 0))

23


However, even for medium size problems, the complexity seems a bit high:

In [2]:
import random, datetime as dt

def generate(n, vmin, vmax):
    # Generate a pyramid of height `n`
    aout = []
    for i in range(n):
        aout.append([random.randint(vmin, vmax) for _ in range(i+1)])
    return aout

print(dt.datetime.now())
print(sum_td(generate(25, 0, 100), 0, 0))
print(dt.datetime.now())

2023-02-13 06:47:59.676157
1816
2023-02-13 06:48:04.751540


This is because we keep resolving the same sub-problems again and again. For example, in the previous example, it is clear that the pyramid at the intersection of the red and blue ones is solved twice: one when solving the blue pyramid, one when solving the red pyramid.

To overcome this problem, we saw in the previous course that we can rely on memoization:

In [3]:
def sum_td_cached(tg, depth, index, cache = None):
    cache = {} if cache is None else cache
    key   = (depth, index)

    if key not in cache:
        if depth+1 == len(tg):
            aout = tg[depth][index]
        else:
            aout  = max(sum_td_cached(tg, depth+1, index  , cache),
                        sum_td_cached(tg, depth+1, index+1, cache))
            aout += tg[depth][index]
        cache[key] = aout

    # print(cache)
    return cache[key]

print(dt.datetime.now())
print(sum_td_cached(generate(25, 0, 100), 0, 0))
print(dt.datetime.now())

2023-02-13 06:48:04.756894
1626
2023-02-13 06:48:04.757242


This is far better! If we look more carefully, we see that solving the full problem reduces to solving a number of subproblems proportional to the number of sub-pyramids, which in turn is quadratic in the height of the pyramid. I.e., we moved from an exponential complexity to a quadratic one.

## Top-down vs bottom-up

When implementing a dynamic programming algorithm, there are two methodologies: either the problem is solved recursively, starting from the initial one and going down. Either we start by solving the sub-problems, and then we progress, solving bigger and bigger problems, up to the point where we reach the final one. Let's exemplify this on the Fibonacci sequence.

A top-down implementation of the Fibonacci sequence (with caching) works as follow:

In [4]:
def fibonacci(n, cache = None):
    cache = {} if cache is None else cache
    if n not in cache:
        if n == 0:
            aout = 0
        elif n == 1:
            aout = 1
        else:
            aout = fibonacci(n-1, cache) + fibonacci(n-2, cache)
        cache[n] = aout
    return cache[n]

print(fibonacci(10))

55


In this implementation, we start directly from the top (here, `10`), and we go down, solving the Fibonacci sequence for `9`, then `8`, etc, etc...

A bottom-up implementation of the Fibonacci sequence works as follow:

In [5]:
def fibonacci_bu(n):
    values = [0, 1]
    for i in range(2, n+1):
        values.append(values[i-1] + values[i-2])
    return values[n]

print(fibonacci_bu(10))

55


Here, we start by computing the value at `0` & `1`, and then we progress, computing the values for `2`, `3`, etc... relying on the previously (and cached) values. We can do the same for the maximum path problem:

In [6]:
def sum_bu(tg):
    cache = [tg[len(tg)-1]]
    for i in range(len(tg)-1):
        line = []
        for j in range(len(cache[i])-1):
            aout = max(cache[i][j], cache[i][j+1])
            line.append(tg[len(tg)-i-2][j] + aout)
        cache.append(line)
    return cache[len(tg)-1][0]

Initially, `cache` is a singleton list, that solves the maximum path problem for all the leaves, i.e., for `EXAMPLE`, cache is set to `[[9, 5, 9, 3]]`. Then, we go up, solving all the subproblems rooted on the penultimate line of the pyramid. Since solving these new sub-problems only require the maximum paths for the last line, we already have all the needed iteration. After one iteration, the `cache` variable is set to `[[9, 5, 9, 3], [11, 13, 15]]`.

And we continue. At the next iteration, we solve all the sub-problems rooted at the second line, obtaining the following value for `cache`: `[[9, 5, 9, 3], [11, 13, 15], [20, 19]]`. Then, the last iteration can solve the initial problem and we have `[[9, 5, 9, 3], [11, 13, 15], [20, 19], [23]]` in `cache`. We simply return the last value (`23`) of the cache.

However, we can see than at each iteration, for resolving the new sub-problems, we only use the last list of `cache`. This means that we are storing too much information and that some memory is wasted. (Note that we had the same problem in `fibonacci_bu`: we do not need to store all the values, only the two last ones are necessary) This means than we can still optimize (memory-wise) our implementations. In the case of Fibonacci, this gives:

In [7]:
def fibonacci_bu_mem(n):
    r0, r1 = 0, 1 # We only store the two last values
    for _ in range(n):
        r0, r1 = r1, r0+r1
    return r0

print(fibonacci_bu_mem(10))

55


And for the maximum path problem, we obtain the following implementation:

In [8]:
def sum_bu_mem(tg):
    cache = tg[len(tg)-1] # We only store the solutions for one line
    for i in range(len(tg)-1):
        line = []
        for j in range(len(cache)-1):
            aout = max(cache[j], cache[j+1])
            line.append(tg[len(tg)-i-2][j] + aout)
        cache = line
    return cache[0]

Note that, when doing so, you must have a clear understanding of the optimal substructure, s.t. you know which sub-solutions you must keep / you can throw away.

Last, we can remark than in all these implementations, we are returning the sum of a maximal path, but we do not return such a path. If we want to do so, we need to store more information. E.g., in the top-down approach, in the cache, we could store a pair that is composed of a maximum path (i.e. a sequence of positions) along with the sum of that path:

In [9]:
def sum_with_path(tg, depth, index, cache = None):
    cache = {} if cache is None else cache
    key   = (depth, index)

    if key not in cache:
        if depth+1 == len(tg):
            aout = ([index], tg[depth][index])
        else:
            li = sum_with_path(tg, depth+1, index  , cache)
            ri = sum_with_path(tg, depth+1, index+1, cache)
            di = li if li[1] > ri[1] else ri
            aout   = ([index] + di[0], tg[depth][index] + di[1])

        cache[key] = aout

    # print(cache)
    return cache[key]

print(sum_with_path(EXAMPLE, 0, 0))

([0, 0, 1, 2], 23)


We could even find all the paths that whose sum are maximal (indeed, there is no unicity of the maximal path):

In [10]:
def sum_with_all_paths(tg, depth, index, cache = None):
    cache = {} if cache is None else cache
    key   = (depth, index)

    if key not in cache:
        if depth+1 == len(tg):
            aout = ([[index]], tg[depth][index])
        else:
            li   = sum_with_all_paths(tg, depth+1, index  , cache)
            ri   = sum_with_all_paths(tg, depth+1, index+1, cache)
            di   = max(li[1], ri[1])
            pi   = sum([x[0] for x in [li, ri] if x[1] == di], [])
            aout = ([[index] + p for p in pi], tg[depth][index] + di)

        cache[key] = aout

    return cache[key]

# Matrix Chain Multiplication

In this problem, we are given a sequence of matrices $[A_i]_{0 \leq i < n}$ and we want to calculate the product $A_{1,n} = A_1 \cdots A_n$ (assuming that the matrices have compatible sizes). There is some freedom in the sequence of the matrix multiplications involved. Indeed, since the matrix multiplication is associative, no matter how the product is parenthesized, the result will remain the same. However, the order in which the matrices are parenthesized has a strong impact on the number of arithmetic operations needed to compute the product. For example, assume that the
number of simple matrix multiplications is of order $k_1 k_2 k_3$  when multiplying two matrices of resp. sizes $k_1 \times k_2$ and $k_2 \times k_3$. Then, for three matrices $A$, $B$, $C$ of respective size $10 \times 50$, $50 \times 3$ and $3 \times 20$:

 - computing $(AB)C$ needs $10 \times 50 \times  3 + 10 \times  3 \times 20 =  2100$ operations, while
 - computing $A(BC)$ needs $50 \times  3 \times 20 + 10 \times 50 \times 20 = 13000$ operations.

The problem here is to determine, before performing any calculation, which parenthesization will require the least simple arithmetic operations.

Let $A_{i,j} = A_i \cdots A_j$ for $0 \leq i \leq j < n$. If, for computing $A_{i,j}$ using an optimal parenthesization, the last multiplication is $(A_i \cdots A_l) \cdot (A_{l+1} \cdots A_j)$, then it must be that we used an optimal parenthesization for computing $A_{i,l}$ and $A_{l+1,j}$. Indeed, if this were not the case, we could replace them by a better parenthesization and thus have a better parenthesization for the product $A_{i,j}$, which is contradictory with the optimality hypothesis we have made. This means that our problem as an optimal substructures. Moreover, it is clear that the same sub-parenthesization can occur in several different parenthesization of the initial problem. These two conditions make it possible to implement dynamic programming techniques.

From now, for $i \in [0 \ldots n[$, let $\alpha_i$ be the number of lines of $A_i$ and $\alpha_{i+1}$ its number of columns. (Note that this definition makes sense because the matrices in $[A_i]_{0 \leq i < n}$ have compatible sizes). For $i \leq j$, we denote by $m_{i,j}$ the number of simple arithmetic operations involved in an optimal parenthesization of $A_{i,j}$. We have:

$$
\left\{ \begin{aligned}
  m_{i, i} &= 0 \\
  m_{i, j}
    &= \min_{i \leq l < j}
         (m_{i,l} + \alpha_i \alpha_{l+1} \alpha_{j+1} + m_{l+1,j})
    & \text{($i < j$)}
\end{aligned} \right.
$$

i.e. for finding the optimal parenthezisation of $A_{i,j}$, for $i < j$, we simply try all the different possibilities for the first multiplication. If this first multiplication is forced to be at position $l$ then, considering the optimal substructure of our problem, the optimal number of operations if equal to the sum of $m_{i,l}$ (the cost of $A_{i,l}$), $m_{l+1,j}$ (the cost of $A_{l+1,j}$) and $\alpha_i \alpha_{l+1} \alpha_{j+1}$ (the cost of the multiplication at $l$). We simply take the minimum among of the possible $l$'s.

This leads to the following implementation that returns the minimum number of arithmetic operations for compution $A_{i,j}$ (here, the argument `a` contains the matrices' sizes, i.e. the $\alpha_i$'s):

In [11]:
def mcm_r(a, i, j, cache):
    if (i, j) in cache:
        return cache[(i, j)]
    
    if i >= j:
        aout = 0
    else:
        aout = min(
            mcm_r(a, i, l, cache)
                + a[i] * a[l+1] * a[j+1]
                + mcm_r(a, l+1, j, cache) 
            for l in range(i, j)
        )

    cache[(i, j)] = aout; return aout
        
def mcm(a):
    return mcm_r(a, 0, len(a)-2, {})

mcm([5, 10, 6, 30, 4, 12, 16])

2228

However, we may want to return the optimal parenthesization along with the minimum number of operations - otherwise, the value we return is quite useless. For that purpose, we are going to store more information in the cache: at position `(i, j)`, we will store the index `l` where the first multiplication must be done when computing $A_{i,j}$. This will then allow us to reconstruct the optimal parenthezisation:

In [12]:
import math

def mcm_c_r(a, i, j, cache):
    if (i, j) in cache:
        return cache[(i, j)][1]
    
    if i >= j:
        cl, cmin = None, 0
    else:
        cl, cmin = None, math.inf
        for l in range(i, j):
            current = mcm_c_r(a, i, l, cache) \
                + a[i] * a[l+1] * a[j+1] \
                + mcm_c_r(a, l+1, j, cache)
            if current < cmin:
                cl, cmin = l, current

    cache[(i, j)] = (cl, cmin); return cmin

def mcm_c(a):
    cache = {}
    mcm_c_r(a, 0, len(a)-2, cache)
    return cache

def reconstruct(cache, i, j):
    if i >= j:
        return "A_{0}".format(i)
    return "({0} {1})".format(
        reconstruct(cache, i, cache[(i, j)][0]),
        reconstruct(cache, cache[(i, j)][0]+1, j),
    )

mtx = [5, 10, 6, 30, 4, 12, 16]
reconstruct(mcm_c(mtx), 0, len(mtx)-2)

'(((A_0 A_1) (A_2 A_3)) (A_4 A_5))'